In [1]:
# KB차차차 중고차 매물 크롤링 - 페이지별 carSeq 추출
import requests
import json
import pandas as pd
from datetime import datetime
import time
from bs4 import BeautifulSoup
import re

BASE_URL = "https://www.kbchachacha.com"

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    'Accept': '*/*',
    'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
    'Host': 'www.kbchachacha.com'
})

def get_car_seqs_from_page(page_num):
    url = f"https://www.kbchachacha.com/public/search/list.empty?page={page_num}&sort=-orderDate"
    
    try:
        res = session.get(url, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # .area 클래스에서 data-car-seq 추출
            areas = soup.select('.area')
            page_car_seqs = []
            
            for area in areas:
                car_seq = area.get('data-car-seq')
                if car_seq:
                    page_car_seqs.append(car_seq)
            
            print(f"페이지 {page_num}: {len(page_car_seqs)}개 carSeq 발견")
            return page_car_seqs
        else:
            print(f"페이지 {page_num}: HTTP {res.status_code}")
            return []
    except Exception as e:
        print(f"페이지 {page_num} 오류: {e}")
        return []

def crawl_multiple_pages(max_pages=5, delay=1):
    """여러 페이지를 크롤링하여 모든 carSeq를 수집합니다."""
    all_car_seqs = []
    
    print(f"KB차차차 {max_pages}페이지 크롤링 시작...")
    
    for page in range(1, max_pages + 1):
        print(f"\n페이지 {page} 처리 중...")
        
        page_car_seqs = get_car_seqs_from_page(page)
        
        if page_car_seqs:
            all_car_seqs.extend(page_car_seqs)
            print(f"페이지 {page} carSeq: {page_car_seqs[:5]}...")  # 처음 5개만 출력
        else:
            print(f"페이지 {page}: carSeq 없음")
            break  # 더 이상 데이터가 없으면 중단
        
        # 서버 부하 방지
        time.sleep(delay)
    
    # 중복 제거
    unique_car_seqs = list(set(all_car_seqs))
    
    print(f"\n=== 크롤링 완료 ===")
    print(f"총 수집된 carSeq: {len(all_car_seqs)}개")
    print(f"중복 제거 후: {len(unique_car_seqs)}개")
    print(f"carSeq 목록: {unique_car_seqs[:10]}...")  # 처음 10개만 출력
    
    return unique_car_seqs

# 크롤링 실행 (테스트용으로 3페이지)
print("KB차차차 페이지별 carSeq 크롤링 시작...")
all_car_seqs = crawl_multiple_pages(max_pages=3, delay=2)

KB차차차 페이지별 carSeq 크롤링 시작...
KB차차차 3페이지 크롤링 시작...

페이지 1 처리 중...
페이지 1: 40개 carSeq 발견
페이지 1 carSeq: ['27306737', '27225812', '27504750', '27491856', '27489612']...

페이지 2 처리 중...
페이지 2: 40개 carSeq 발견
페이지 2 carSeq: ['27493386', '27158527', '27144129', '27142532', '27096462']...

페이지 3 처리 중...
페이지 3: 40개 carSeq 발견
페이지 3 carSeq: ['27249349', '27488831', '27464455', '27372648', '27468605']...

=== 크롤링 완료 ===
총 수집된 carSeq: 120개
중복 제거 후: 119개
carSeq 목록: ['27321627', '27301450', '27153421', '27480719', '27097919', '27024311', '27345737', '27355976', '27464566', '27185606']...


In [2]:
# 🚀 더 효율적인 방법: API를 통한 일괄 정보 수집
def get_car_info_via_api(car_seqs):
    """carSeq 리스트를 API로 보내서 일괄로 차량 정보를 받아옵니다."""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:
        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            return data.get('list', [])
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트: API를 통한 차량 정보 수집 (샘플 carSeq 사용)
test_car_seqs = ['27490092', '27483488', '27471263']  # 테스트용 carSeq

print(f"\n{'='*60}")
print("🚀 API를 통한 효율적인 차량 정보 수집 테스트")
print(f"{'='*60}")

car_info_list = get_car_info_via_api(test_car_seqs)

if car_info_list:
    print(f"\n=== 수집된 차량 정보 (상세) ===")
    for i, car in enumerate(car_info_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 가격 정보 (priceMin, priceMax, priceRateMin, priceRateMax, priceRateAvg)
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        # 연식 (yymm)
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        # 등록일 (regiDay)
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        # 제조사명 (makerName)
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        # 차량 사진 (photoName1)
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 API 필드들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # API에서 사용 가능한 모든 필드 출력 (디버깅용)
        if i == 1:  # 첫 번째 차량만 전체 필드 출력
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ API를 통한 정보 수집에 실패했습니다.")



🚀 API를 통한 효율적인 차량 정보 수집 테스트

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (상세) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 20231006
      yymm: 2023
      km: 19707


In [3]:
# 🚀 제조사 정보를 통한 국산/수입 구분 시스템
def get_maker_info():
    """제조사 정보를 가져와서 makerCode와 countryCode 매핑을 생성합니다."""
    
    maker_url = "https://www.kbchachacha.com/public/search/carMaker.json?page=1&sort=-orderDate"
    
    try:
        response = requests.get(maker_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            maker_info = {}
            
            # 국산차 정보
            for maker in data.get('result', {}).get('국산', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            # 수입차 정보
            for maker in data.get('result', {}).get('수입', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            print(f"✅ 제조사 정보 수집 완료: 총 {len(maker_info)}개 제조사")
            return maker_info
        else:
            print(f"❌ 제조사 정보 수집 실패: HTTP {response.status_code}")
            return {}
            
    except Exception as e:
        print(f"❌ 제조사 정보 수집 오류: {e}")
        return {}

# 제조사 정보 수집
maker_info = get_maker_info()

# 제조사 정보 출력 (처음 10개만)
if maker_info:
    print(f"\n📋 제조사 정보 샘플:")
    for i, (code, info) in enumerate(list(maker_info.items())[:10]):
        print(f"   {code}: {info['makerName']} ({info['countryCode']}) - {info['count']}대")


✅ 제조사 정보 수집 완료: 총 82개 제조사

📋 제조사 정보 샘플:
   101: 현대 (국산) - 49382대
   189: 제네시스 (국산) - 7385대
   102: 기아 (국산) - 41970대
   103: 한국GM (국산) - 9297대
   105: 르노코리아 (국산) - 7725대
   104: KG모빌리티 (국산) - 7247대
   106: 기타 (국산) - 178대
   108: 벤츠 (수입) - 8421대
   107: BMW (수입) - 8373대
   109: 아우디 (수입) - 2803대


In [10]:
# 🚀 간단한 API 테스트 - session 문제 완전 해결
import requests

def get_car_info_simple(car_seqs):
    """간단한 API 호출 함수"""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:
        # session 생성
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
            'Host': 'www.kbchachacha.com'
        })
        
        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            return data.get('list', [])
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"\n{'='*60}")
print("🚀 간단한 API 테스트")
print(f"{'='*60}")

car_list = get_car_info_simple(test_car_seqs)

# 결과 출력
if car_list:
    print(f"\n=== 수집된 차량 정보 ===")
    for i, car in enumerate(car_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 요청하신 필드들
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        
        # 첫 번째 차량만 전체 필드 출력
        if i == 1:
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ 차량 정보 수집에 실패했습니다.")



🚀 간단한 API 테스트

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 20231006
      yymm: 2023
      km: 19707
      sellAmtGbn: 109100
      sellAmt: 1630
      ownerYn: Y
      cityName: 경기
      photoName1: 27490092_4656596854618635.jpeg
      wishYn: N
      co

In [ ]:
def get_maker_info():
    """제조사 정보를 가져와서 국산/수입 구분을 위한 매핑을 생성합니다."""
    
    maker_url = "https://www.kbchachacha.com/public/search/carMaker.json?page=1&sort=-orderDate"
    
    try:
        response = requests.get(maker_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            maker_info = {}
            
            # 국산차 정보
            for maker in data.get('result', {}).get('국산', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            # 수입차 정보
            for maker in data.get('result', {}).get('수입', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            print(f"✅ 제조사 정보 수집 완료: 총 {len(maker_info)}개 제조사")
            return maker_info
        else:
            print(f"❌ 제조사 정보 수집 실패: HTTP {response.status_code}")
            return {}
            
    except Exception as e:
        print(f"❌ 제조사 정보 수집 오류: {e}")
        return {}

# 제조사 정보 수집
print("📋 제조사 정보 수집 중...")
maker_info = get_maker_info()

# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"\n{'='*60}")
print("🚀 완전한 KB차차차 크롤링 시스템")
print(f"{'='*60}")

car_list = get_car_info_simple(test_car_seqs)

# 결과 출력
if car_list:
    print(f"\n=== 수집된 차량 정보 (완전 버전) ===")
    for i, car in enumerate(car_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 요청하신 필드들
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 국산/수입 구분 (maker_info가 있는 경우)
        if maker_info:
            maker_code = car.get('makerCode', '')
            if maker_code in maker_info:
                country_code = maker_info[maker_code]['countryCode']
                print(f"   countryCode: {country_code}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 정보들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # 첫 번째 차량만 전체 필드 출력
        if i == 1:
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ 차량 정보 수집에 실패했습니다.")


📋 제조사 정보 수집 중...
✅ 제조사 정보 수집 완료: 총 82개 제조사

🚀 완전한 KB차차차 크롤링 시스템

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (완전 버전) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   countryCode: 국산
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지


In [12]:
# 🚀 향상된 API 정보 수집 - 국산/수입 구분 포함
def get_car_info_via_api_enhanced(car_seqs, maker_info_dict):
    """carSeq 리스트를 API로 보내서 일괄로 차량 정보를 받아옵니다. (국산/수입 구분 포함)"""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:

        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            # 차량 정보에 국산/수입 정보 추가
            car_list = data.get('list', [])
            for car in car_list:
                maker_code = car.get('makerCode', '')
                if maker_code in maker_info_dict:
                    car['countryCode'] = maker_info_dict[maker_code]['countryCode']
                    car['makerName_kor'] = maker_info_dict[maker_code]['makerName']
                else:
                    car['countryCode'] = '알수없음'
                    car['makerName_kor'] = car.get('makerName', '')
            
            return car_list
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트: 향상된 API를 통한 차량 정보 수집
test_car_seqs = ['27490092', '27483488', '27471263']  # 테스트용 carSeq

print(f"\n{'='*60}")
print("🚀 향상된 API를 통한 차량 정보 수집 테스트 (국산/수입 구분)")
print(f"{'='*60}")

# maker_info가 정의되지 않은 경우 빈 딕셔너리 사용
if 'maker_info' not in globals():
    print("⚠️ maker_info가 정의되지 않았습니다. 빈 딕셔너리를 사용합니다.")
    maker_info = {}

car_info_list = get_car_info_via_api_enhanced(test_car_seqs, maker_info)

if car_info_list:
    print(f"\n=== 수집된 차량 정보 (국산/수입 구분 포함) ===")
    for i, car in enumerate(car_info_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 국산/수입 구분
        country_code = car.get('countryCode', '알수없음')
        print(f"   countryCode: {country_code}")
        
        # 가격 정보 (priceMin, priceMax, priceRateMin, priceRateMax, priceRateAvg)
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        # 연식 (yymm)
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        # 등록일 (regiDay)
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        # 제조사명 (makerName)
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        # 차량 사진 (photoName1)
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 API 필드들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # API에서 사용 가능한 모든 필드 출력 (디버깅용)
        if i == 1:  # 첫 번째 차량만 전체 필드 출력
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ API를 통한 정보 수집에 실패했습니다.")



🚀 향상된 API를 통한 차량 정보 수집 테스트 (국산/수입 구분)

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (국산/수입 구분 포함) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   countryCode: 국산
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 2023